# SDSS SQL exercises

1. Find the _ugriz_ magnitudes (in `PhotoObj table`) measured for all QSOs in SDSS DR18. How many QSO did you find? How can you get the table with the magnitudes if SDSS limitation for tables with measured parameters is 500,000?

In [1]:
from astroquery.sdss import SDSS

In [2]:
query1 = """
SELECT Count(*)
FROM PhotoObj AS p
JOIN SpecObj AS s ON s.bestobjid = p.objid
WHERE 
  p.u > 0 AND p.g > 0 AND p.r > 0 AND p.i > 0 AND p.z > 0
  AND s.Class = 'QSO'
    
"""

In [3]:
QSO_df = SDSS.query_sql(query1).to_pandas()
QSO_df

,Column1
0,866319


In [4]:
query2 = """
SELECT Count(*)
FROM PhotoObj AS p
JOIN SpecObj AS s ON s.bestobjid = p.objid
WHERE 
  p.u > 0 AND p.g > 0 AND p.r > 0 AND p.i > 0 AND p.z > 0
  AND s.Class = 'QSO'
  AND s.ra < 180
    
"""
QSO2_df = SDSS.query_sql(query2).to_pandas()
QSO2_df

,Column1
0,461820


In [5]:
query3 = """
 SELECT Count(*)
FROM PhotoObj AS p
JOIN SpecObj AS s ON s.bestobjid = p.objid
WHERE 
  p.u > 0 AND p.g > 0 AND p.r > 0 AND p.i > 0 AND p.z > 0
  AND s.Class = 'QSO'
  AND s.ra > 180
    
"""
QSO3_df = SDSS.query_sql(query3).to_pandas()
QSO3_df

,Column1
0,404499


2. Find all starburst galaxies in SDSS DR18, with high quality spectra (S/N median > 40, without warning for redshift), and with velocity dispersion of Hbeta>100 km/s (`Emission Lines Port`). How many starburst galaxies did you find? Download these spectra in fits format. Caution: in order to directly download them, you need to put as output in SQL their plate, mjd and fiber.

In [6]:
query4 = """
SELECT s.plate, s.mjd, s.fiberID

FROM SpecObj AS s
    JOIN emissionLinesPort AS e ON s.specObjID = e.specObjID
    
WHERE s.subclass = 'STARBURST'
    AND s.snMedian > 40.0
    AND e.Sigma_Hb_4861 > 100.0
    AND s.Zwarning = 0.0
    
"""

starburst_df = SDSS.query_sql(query4).to_pandas()
starburst_df

,plate,mjd,fiberID
0,952,52409,247
1,1808,54176,301
2,1301,52976,25
3,630,52050,245
4,491,51942,278
5,1018,52672,22
6,1622,53385,113
7,2112,53534,555
8,2091,53447,571
9,891,52584,2


3. Find all galaxies in SDSS DR18, with g magnitude < 15, S/N median>10, and EW Ha 6562>10 (`EmissionLinesPort`). How many galaxies did you find? Download their spectra.

In [7]:
query5 = """
SELECT Count(*)

FROM PhotoObj AS p
JOIN SpecObj AS s ON s.bestobjid = p.objid

JOIN emissionLinesPort AS e ON s.specObjID= e.specObjID
    
WHERE s.class = 'GALAXY'
AND p.g < 15
    AND s.snMedian > 10.0
    AND e.EW_Ha_6562 > 10
    
"""

galaxy_df = SDSS.query_sql(query5).to_pandas()
galaxy_df

,Column1
0,1575


In [8]:
query5 = """
SELECT s.plate, s.mjd, s.fiberID

FROM PhotoObj AS p
JOIN SpecObj AS s ON s.bestobjid = p.objid

JOIN emissionLinesPort AS e ON s.specObjID= e.specObjID
    
WHERE s.class = 'GALAXY'
AND p.g < 15
    AND s.snMedian > 10.0
    AND e.EW_Ha_6562 > 10
    
"""

galaxy_df = SDSS.query_sql(query5).to_pandas()
galaxy_df

,plate,mjd,fiberID
0,288,52000,257
1,298,51955,189
2,504,52316,95
3,418,51817,493
4,428,51883,482
...,...,...,...
1570,2596,54207,207
1571,2606,54154,518
1572,665,52168,377
1573,853,52374,184


In [9]:
### Let's make a function that generates wget links

def get_wget_url(plate, mjd, fiber):
    url = 'https://data.sdss.org/sas/dr18/spectro/sdss/redux'
    
    if mjd > 55000:
        url += '/v5_13_2/'
    else:
        url += '/26/'
        
    url += 'spectra/lite/{plate:04}/spec-{plate:04}-{mjd:05}-{fiber:04}.fits'.format(plate=plate, mjd=mjd, fiber=fiber)
    return url